In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!nvidia-smi

Mon May  3 02:34:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |   1437MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-03 02:34:10--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-03 02:34:10--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc700dc4761c648d31f4b23c1005.dl.dropboxusercontent.com/cd/0/inline/BNzQoVvBmap8trO3JSbT1C54QtHSLv4lxwh7AAdRQ1f2RLCpCYAOVfAQcgXQu8epqznMmhrNA8wKza0KjzQ_bczHu_Fdh9hjXbOCSBTZqwh9vp-ziM88KSVcuUh6OyPRKsGreCA3zdiSnp7Auh79jBsn/file# [following]
--2021-05-03 02:34:10--  https://uc700dc4761c648d31f4b23c1005.dl.dropboxusercontent.com/cd/0/inline/BNzQoVvBmap8trO3JSbT1C54QtHSLv4lxwh7AAdRQ1f2RLCpCYAOVfAQcgXQu

In [ ]:
import pickle
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [ ]:
data = pd.read_pickle(r'grouped_data.pickle')

In [ ]:
texts = []
bos = "<BOS> "
eos = " <EOS> "

for key in sorted(list(data.keys())):
    post = data[key][0]
    comments = data[key][1]
    title = post['title']
    # try:
    #     text = post['selftext']
    # except:
    #     text = ""
    
    for comment in comments:
        if len(title) < 500 and len(comment['body']) < 500 and comment['body'] != '[deleted]' and comment['body'] != '[removed]':
            texts.append(title + bos + comment["body"]  + eos )

train_sample, test_sample = train_test_split(texts[280000*4:], test_size=0.25, shuffle=True, stratify=None)

print(len(train_sample), len(test_sample))

with open('train_text.txt', 'w', encoding='utf8') as f:
    for item in train_sample:
        f.write("%s\n" % item)

with open('test_text.txt', 'w', encoding='utf8') as f:
    for item in test_sample:
        f.write("%s\n" % item)

212515 70839


In [ ]:
len(texts)

1403354

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_text.txt'
test_path = 'test_text.txt'

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
print('Size of training set:', len(train_dataset))
print('Size of test set:', len(test_dataset))

Size of training set: 92894
Size of test set: 30984


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:


model = AutoModelWithLMHead.from_pretrained("/content/gdrive/My Drive/gpt_title_only_0.8")


training_args = TrainingArguments(
    output_dir="./gpt2-CC", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler,
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [ ]:
trainer.train()

Step,Training Loss
500,2.233400
1000,2.065300
1500,2.095000
2000,2.111900
2500,2.132100
3000,2.149400
3500,2.168500
4000,2.191000
4500,2.198000
5000,2.216500


TrainOutput(global_step=11612, training_loss=2.197008133591597, metrics={'train_runtime': 4067.9267, 'train_samples_per_second': 2.855, 'total_flos': 1.7755716901404672e+16, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 16384, 'init_mem_gpu_alloc_delta': 514529280, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -515387392, 'train_mem_gpu_alloc_delta': 1508786688, 'train_mem_cpu_peaked_delta': 516616192, 'train_mem_gpu_peaked_delta': 4969645056})

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model('/content/gdrive/My Drive/gpt_title_only_1')

# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
from transformers import pipeline

chef = pipeline('text-generation',model='/content/gdrive/My Drive/gpt_title_only_0.4', tokenizer='gpt2',config={'max_length':1500})



In [ ]:
chef("Will Sophia do good on her exam tomorrow? <BOS> ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Will Sophia do good on her exam tomorrow? <BOS> \nSongs that get stuck in your head?<BOS> [Cinema by Gorillaz and the Black Keys.](https://www.youtube.com/'}]